In [1]:
import triton
??triton.runtime.autotuner.Autotuner

Init signature:
triton.runtime.autotuner.Autotuner(
    fn,
    arg_names,
    configs,
    key,
    reset_to_zero,
    restore_value,
    pre_hook=None,
    post_hook=None,
    prune_configs_by: 'Dict' = None,
    warmup=None,
    rep=None,
    use_cuda_graph=False,
    do_bench=None,
)
Docstring:     
Abstract base class for generic types.

A generic type is typically declared by inheriting from
this class parameterized with one or more type variables.
For example, a generic mapping type might be defined as::

  class Mapping(Generic[KT, VT]):
      def __getitem__(self, key: KT) -> VT:
          ...
      # Etc.

This class can then be used as follows::

  def lookup_name(mapping: Mapping[KT, VT], key: KT, default: VT) -> VT:
      try:
          return mapping[key]
      except KeyError:
          return default
Source:        
class Autotuner(KernelInterface):

    def __init__(
        self,
        fn,
        arg_names,
        configs,
        key,
        reset_to_zero,
     

In [ ]:
from tests.test_attention import flash_backward_results, _attention_and_lse, _make_attn_inputs
from cs336_systems.flashattention import FlashAttentionTriton
# from cs336_systems.flashattention_compiled import FlashAttentionTritonCompiled
from cs336_systems.flashattention_triton_optimized import FlashAttentionTritonOptimized

# impl_0 = AttentionAutogradFunctionPytorch.apply
# q, k, v, do = _make_attn_inputs(device='cuda')
# o_0 = impl_0(q, k, v, True)
# o_0.backward(do)
# v.grad

# import torch
# torch.testing.assert_close(o_0, o_1, rtol=1e-2, atol=1e-2)

In [2]:
impl_0 = FlashAttentionTriton.apply
q, k, v, do = _make_attn_inputs(device='cuda')
o_0 = impl_0(q, k, v, True)
o_0;

In [3]:
impl_1 = FlashAttentionTritoneOptimized.apply
q, k, v, do = _make_attn_inputs(device='cuda')
o_1 = impl_1(q, k, v, True)
o_1;

In [5]:
class FlashAttention2:
    def __init__(self, impl, B_q=16, B_k=16):
        self.impl = impl
        self.B_q = B_q
        self.B_k = B_k
    
    def __call__(self, Q, K, V, is_causal=True):
        return self.impl.apply(Q, K, V, is_causal, self.B_q, self.B_k)

# # Usage:
# flash_small = FlashAttentionPytorch(B_q=8, B_k=8)
# o = flash_small(q, k, v, False)

In [ ]:
import torch
import triton
def test_timing_flash_forward_backward(test, impl, n_heads, d_head, sequence_length, dtype=torch.bfloat16, device='cuda', B_q=16, B_k=16):
    q, k, v = torch.randn(
        3, n_heads, sequence_length, d_head, device=device, dtype=dtype, requires_grad=True
    )
    
    flash = torch.compile(FlashAttention2(impl, B_q, B_k))
    # sanity check; it would fail without compiling if precision in triton is not implemented right
    # flash = FlashAttention2(impl, B_q, B_k)
    
    def flash_forward():
        o = flash(q, k, v, True)

    def flash_forward_backward():
        o = flash(q, k, v, True)
        loss = o.sum()
        loss.backward()

    if test == "forward":
        results = triton.testing.do_bench(flash_forward, rep=1000, warmup=1000)
    elif test == "forward_backward":
        results = triton.testing.do_bench(flash_forward_backward, rep=1000, warmup=1000)
    else:
        raise ValueError("Wrong selection.")
    print(results)

In [ ]:
test_timing_flash_forward_backward("forward_backward", FlashAttentionAutogradFunctionTriton, 16, 128, 16384, dtype=torch.bfloat16)

In [ ]:
test_timing_flash_forward_backward("forward_backward", FlashAttentionAutogradFunctionTritonBackward, 16, 128, 16384, dtype=torch.bfloat16, B_q=16, B_k=16)